# Create a basic working model (no hyperparameterization) 

### Split Locations for reference 
#### SCAFFOLD: 
Dataset split table saved to /home/jupyter/hERG_Analysis/herg_chembl_curated_train_valid_test_scaffold_5f834022-46ac-4c7b-bca1-9573f6eb940f.csv

#### RANDOM:
Dataset split table saved to /home/jupyter/hERG_Analysis/herg_chembl_curated_train_valid_test_random_ac01fe91-0793-4566-9340-2ff90e6a58c5.csv

#### FINGERPRINT:
Dataset split table saved to /home/jupyter/hERG_Analysis/herg_chembl_curated_train_valid_test_fingerprint_64ffab14-47a7-44e8-bfa1-8576f640e241.csv

In [6]:
# import packages needed 

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set_context("poster")
sns.set_style("whitegrid")
sns.set_palette("Set2")

import pandas as pd
import os, json, sys, glob, pickle

from atomsci.ddm.pipeline import perf_data

# We temporarily disable warnings for demonstration.
# FutureWarnings and DeprecationWarnings are present from some of the AMPL 
# dependency modules.
import warnings
warnings.filterwarnings('ignore')



import pandas as pd
import os, json, sys, glob, pickle
import requests


from atomsci.ddm.pipeline import model_pipeline as mp
from atomsci.ddm.pipeline import parameter_parser as parse
import atomsci.ddm.utils.curate_data as curate_data
import atomsci.ddm.utils.struct_utils as struct_utils
from atomsci.ddm.pipeline import perf_plots as pp
import atomsci.ddm.pipeline.diversity_plots as dp

# Random Forest

## Random Forest with Scaffold Split

In [16]:
response_col = "class"
compound_id = "compound_id"
smiles_col = "rdkit_smiles"
split_uuid = "5f834022-46ac-4c7b-bca1-9573f6eb940f" # scaffold
# Set up file locations 
dataset_file = '/home/jupyter/hERG_Analysis/herg_chembl_curated.csv' 
odir = '/home/jupyter/hERG_Analysis/Models' 

params = {
        "verbose": "True",
        "datastore": "False",
        "save_results": "False",
        "prediction_type": "classification",
        "dataset_key": dataset_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        # using previously split data
        "previously_split": "True",
        "split_uuid" : split_uuid,
        "split_only": "False",
        # not needed becuase already using splits established previously
        # "split_strategy": "train_valid_test",
        # "splitter": "fingerprint",
        # "split_valid_frac": "0.10",
        # "split_test_frac": "0.10",
        "featurizer": "ecfp",
        "model_type": "RF",
        "verbose": "True",
        "transformers": "True",
        'max_epochs': '70',
        "rerun": "False",
        "result_dir": odir,
    #    "rf_estimators": 64,
     #   "rf_max_depth": 24,
      #  "rf_max_features": 16,
       # "model_choice_score_type": "recall"
        # "ampl_version": " "
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

2024-04-19 12:19:04,379 Previous dataset split restored


In [20]:
pred_data_valid = pl.model_wrapper.get_perf_data(subset="valid", epoch_label="best")
pred_results_valid = pred_data_valid.get_prediction_results()
pred_results_valid
#print(f"AUC score of validation set: {pred_results['prc_auc_score']:.3f}")

{'roc_auc_score': 0.7807881631346585,
 'prc_auc_score': 0.7454353527415902,
 'cross_entropy': 0.5668310706801162,
 'model_choice_score': 0.7807881631346585,
 'precision': 0.7054108216432866,
 'recall_score': 0.6506469500924215,
 'npv': 0.7232796486090776,
 'accuracy_score': 0.7157360406091371,
 'bal_accuracy': 0.7106588884627474,
 'kappa': 0.42388469338315715,
 'matthews_cc': 0.4249881362060252,
 'confusion_matrix': [[[494, 147], [189, 352]]],
 'num_compounds': 1182}

In [23]:
pred_data_test = pl.model_wrapper.get_perf_data(subset="test", epoch_label="best")
pred_results_test = pred_data_test.get_prediction_results()
pred_results_test

{'roc_auc_score': 0.7682053685334166,
 'prc_auc_score': 0.7348075363847076,
 'cross_entropy': 0.5694910353659588,
 'precision': 0.6948051948051948,
 'recall_score': 0.6045197740112994,
 'npv': 0.7083333333333334,
 'accuracy_score': 0.7030456852791879,
 'bal_accuracy': 0.6939649561300737,
 'kappa': 0.3926322169227121,
 'matthews_cc': 0.39546111547086926,
 'confusion_matrix': [[[510, 141], [210, 321]]],
 'num_compounds': 1182}

In [8]:
# Model Performance
from atomsci.ddm.pipeline import compare_models as cm
rf_s_pred_df = cm.get_filesystem_perf_results(odir, pred_type='classification')

2024-04-18 16:28:05,085 Warning: column names have been changed to align with get_multitask_perf_from_tracker(): featurizer is now features and <subset>_<metric> has been changed to best_<subset>_<metric>.


Found data for 1 models under /home/jupyter/hERG_Analysis/Models


In [12]:
# View the pred_df dataframe
rf_s_pred_df.columns

Index(['model_uuid', 'model_path', 'ampl_version', 'model_type', 'dataset_key',
       'features', 'splitter', 'split_strategy', 'model_score_type',
       'feature_transform_type', 'model_choice_score',
       'best_train_roc_auc_score', 'best_train_prc_auc_score',
       'best_train_precision', 'best_train_recall_score',
       'best_train_num_compounds', 'best_train_accuracy_score',
       'best_train_bal_accuracy', 'best_train_npv', 'best_train_matthews_cc',
       'best_train_kappa', 'best_train_cross_entropy',
       'best_train_confusion_matrix', 'best_valid_roc_auc_score',
       'best_valid_prc_auc_score', 'best_valid_precision',
       'best_valid_recall_score', 'best_valid_num_compounds',
       'best_valid_accuracy_score', 'best_valid_bal_accuracy',
       'best_valid_npv', 'best_valid_matthews_cc', 'best_valid_kappa',
       'best_valid_cross_entropy', 'best_valid_confusion_matrix',
       'best_test_roc_auc_score', 'best_test_prc_auc_score',
       'best_test_precision', 

In [14]:
# Top performing model
top_model=rf_s_pred_df.sort_values(by="best_valid_roc_auc_score", ascending=False).iloc[0]
top_model

model_uuid                            804a2c09-754c-4dc4-986f-273eea98a9be
model_path               /home/jupyter/hERG_Analysis/Models/herg_chembl...
ampl_version                                                         1.6.0
model_type                                                              RF
dataset_key              /home/jupyter/hERG_Analysis/herg_chembl_curate...
                                               ...                        
xgb_subsample                                                          NaN
xgb_n_estimators                                                       NaN
xgb_min_child_weight                                                   NaN
model_parameters_dict    {"rf_estimators": 500, "rf_max_depth": null, "...
feat_parameters_dict                 {"ecfp_radius": 2, "ecfp_size": 1024}
Name: 0, Length: 64, dtype: object

## Random Forest with Fingerprint Split

In [25]:
response_col = "class"
compound_id = "compound_id"
smiles_col = "rdkit_smiles"
split_uuid = "64ffab14-47a7-44e8-bfa1-8576f640e241" # fingerprint
dataset_file = '/home/jupyter/hERG_Analysis/herg_chembl_curated.csv' 
odir = '/home/jupyter/hERG_Analysis/Models' 

params = {
        "verbose": "True",
        "datastore": "False",
        "save_results": "False",
        "prediction_type": "classification",
        "dataset_key": dataset_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        # using previously split data
        "previously_split": "True",
        "split_uuid" : split_uuid,
        "split_only": "False",
        # not needed becuase already using splits established previously
        # "split_strategy": "train_valid_test",
        "splitter": "fingerprint",
        # "split_valid_frac": "0.10",
        # "split_test_frac": "0.10",
        "featurizer": "ecfp",
        "model_type": "RF",
        "verbose": "True",
        "transformers": "True",
        'max_epochs': '70',
        "rerun": "False",
        "result_dir": odir,
    #    "rf_estimators": 64,
     #   "rf_max_depth": 24,
      #  "rf_max_features": 16,
       # "model_choice_score_type": "recall"
        # "ampl_version": " "
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

2024-04-19 12:23:09,790 Previous dataset split restored


In [26]:
pred_data_valid = pl.model_wrapper.get_perf_data(subset="valid", epoch_label="best")
pred_results_valid = pred_data_valid.get_prediction_results()
pred_results_valid

{'roc_auc_score': 0.7477041331372041,
 'prc_auc_score': 0.7691853095599701,
 'cross_entropy': 0.6219622822459739,
 'model_choice_score': 0.7477041331372041,
 'precision': 0.7223168654173765,
 'recall_score': 0.6677165354330709,
 'npv': 0.6447811447811448,
 'accuracy_score': 0.6833192209991532,
 'bal_accuracy': 0.6845908684491362,
 'kappa': 0.3669212202519163,
 'matthews_cc': 0.36813839926988023,
 'confusion_matrix': [[[383, 163], [211, 424]]],
 'num_compounds': 1181}

In [27]:
pred_data_test = pl.model_wrapper.get_perf_data(subset="test", epoch_label="best")
pred_results_test = pred_data_test.get_prediction_results()
pred_results_test

{'roc_auc_score': 0.6463883841104291,
 'prc_auc_score': 0.5094390513763931,
 'cross_entropy': 0.6550118843469932,
 'precision': 0.5413333333333333,
 'recall_score': 0.4211618257261411,
 'npv': 0.6547029702970297,
 'accuracy_score': 0.6187658495350803,
 'bal_accuracy': 0.5878990298388195,
 'kappa': 0.1821130683655997,
 'matthews_cc': 0.18564159502814637,
 'confusion_matrix': [[[529, 172], [279, 203]]],
 'num_compounds': 1183}

## Random Forest with Random Split

In [28]:
response_col = "class"
compound_id = "compound_id"
smiles_col = "rdkit_smiles"
split_uuid = "ac01fe91-0793-4566-9340-2ff90e6a58c5" # random
# Set up file locations 
dataset_file = '/home/jupyter/hERG_Analysis/herg_chembl_curated.csv' 
odir = '/home/jupyter/hERG_Analysis/Models' 

params = {
        "verbose": "True",
        "datastore": "False",
        "save_results": "False",
        "prediction_type": "classification",
        "dataset_key": dataset_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        # using previously split data
        "previously_split": "True",
        "split_uuid" : split_uuid,
        "split_only": "False",
        # not needed becuase already using splits established previously
        # "split_strategy": "train_valid_test",
        "splitter": "random",
        # "split_valid_frac": "0.10",
        # "split_test_frac": "0.10",
        "featurizer": "ecfp",
        "model_type": "RF",
        "verbose": "True",
        "transformers": "True",
        'max_epochs': '70',
        "rerun": "False",
        "result_dir": odir,
        "rf_estimators": 64,
        "rf_max_depth": 24,
        "rf_max_features": 16,
        #"model_choice_score_type": "recall"
        #"ampl_version": "1.6"
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

2024-04-19 12:24:18,675 Previous dataset split restored


In [29]:
pred_data_valid = pl.model_wrapper.get_perf_data(subset="valid", epoch_label="best")
pred_results_valid = pred_data_valid.get_prediction_results()
pred_results_valid

{'roc_auc_score': 0.8457492498339326,
 'prc_auc_score': 0.8462134226121827,
 'cross_entropy': 0.522644335806418,
 'model_choice_score': 0.8457492498339326,
 'precision': 0.783068783068783,
 'recall_score': 0.7449664429530202,
 'npv': 0.7528455284552845,
 'accuracy_score': 0.7673434856175972,
 'bal_accuracy': 0.7675344160157592,
 'kappa': 0.5348467806331407,
 'matthews_cc': 0.5354914049134292,
 'confusion_matrix': [[[463, 123], [152, 444]]],
 'num_compounds': 1182}

In [30]:
pred_data_test = pl.model_wrapper.get_perf_data(subset="test", epoch_label="best")
pred_results_test = pred_data_test.get_prediction_results()
pred_results_test

{'roc_auc_score': 0.8217382198352039,
 'prc_auc_score': 0.8022574301469595,
 'cross_entropy': 0.5417538685590604,
 'precision': 0.7308377896613191,
 'recall_score': 0.7269503546099291,
 'npv': 0.7520128824476651,
 'accuracy_score': 0.7419627749576988,
 'bal_accuracy': 0.7413068925153206,
 'kappa': 0.48272596571881154,
 'matthews_cc': 0.48273221403911487,
 'confusion_matrix': [[[467, 151], [154, 410]]],
 'num_compounds': 1182}

# Neural Network

## Neural Network with Scaffold Split

In [31]:
response_col = "class"
compound_id = "compound_id"
smiles_col = "rdkit_smiles"
split_uuid = "5f834022-46ac-4c7b-bca1-9573f6eb940f" # scaffold
# Set up file locations 
dataset_file = '/home/jupyter/hERG_Analysis/herg_chembl_curated.csv' 
odir = '/home/jupyter/hERG_Analysis/Models' 

params = {
        "verbose": "True",
        "datastore": "False",
        "save_results": "False",
        "prediction_type": "classification",
        "dataset_key": dataset_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        # using previously split data
        "previously_split": "True",
        "split_uuid" : split_uuid,
        "split_only": "False",
        # not needed becuase already using splits established previously
        # "split_strategy": "train_valid_test",
        # "splitter": "fingerprint",
        # "split_valid_frac": "0.10",
        # "split_test_frac": "0.10",
        "featurizer": "ecfp",
        "model_type": "NN",
        "verbose": "True",
        "transformers": "True",
        'max_epochs': '70',
        "rerun": "False",
        "result_dir": odir,
    #    "rf_estimators": 64,
     #   "rf_max_depth": 24,
      #  "rf_max_features": 16,
       # "model_choice_score_type": "recall"
        # "ampl_version": " "
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

2024-04-19 12:27:39,981 Previous dataset split restored


['/home/jupyter/hERG_Analysis/Models/herg_chembl_curated/NN_ecfp_scaffold_classification/5de4bfcc-33c9-490b-b02e-9745fe59d0d4/model/checkpoint1.pt', '/home/jupyter/hERG_Analysis/Models/herg_chembl_curated/NN_ecfp_scaffold_classification/5de4bfcc-33c9-490b-b02e-9745fe59d0d4/model/checkpoint2.pt', '/home/jupyter/hERG_Analysis/Models/herg_chembl_curated/NN_ecfp_scaffold_classification/5de4bfcc-33c9-490b-b02e-9745fe59d0d4/model/checkpoint3.pt', '/home/jupyter/hERG_Analysis/Models/herg_chembl_curated/NN_ecfp_scaffold_classification/5de4bfcc-33c9-490b-b02e-9745fe59d0d4/model/checkpoint4.pt', '/home/jupyter/hERG_Analysis/Models/herg_chembl_curated/NN_ecfp_scaffold_classification/5de4bfcc-33c9-490b-b02e-9745fe59d0d4/model/checkpoint5.pt']
/home/jupyter/hERG_Analysis/Models/herg_chembl_curated/NN_ecfp_scaffold_classification/5de4bfcc-33c9-490b-b02e-9745fe59d0d4/model/checkpoint1.pt


In [32]:
pred_data_valid = pl.model_wrapper.get_perf_data(subset="valid", epoch_label="best")
pred_results_valid = pred_data_valid.get_prediction_results()
pred_results_valid

{'roc_auc_score': 0.7529939644905574,
 'prc_auc_score': 0.7007429624578451,
 'cross_entropy': 1.7854981222721624,
 'model_choice_score': 0.7529939644905574,
 'precision': 0.6451612903225806,
 'recall_score': 0.7024029574861368,
 'npv': 0.7284991568296796,
 'accuracy_score': 0.6869712351945855,
 'bal_accuracy': 0.688174957682226,
 'kappa': 0.3737631772633677,
 'matthews_cc': 0.37500277020416317,
 'confusion_matrix': [[[432, 209], [161, 380]]],
 'num_compounds': 1182}

In [33]:
pred_data_test = pl.model_wrapper.get_perf_data(subset="test", epoch_label="best")
pred_results_test = pred_data_test.get_prediction_results()
pred_results_test

{'roc_auc_score': 0.7532855436081243,
 'prc_auc_score': 0.6991676934000786,
 'cross_entropy': 1.6936392539673484,
 'precision': 0.6481149012567325,
 'recall_score': 0.67984934086629,
 'npv': 0.728,
 'accuracy_score': 0.6903553299492385,
 'bal_accuracy': 0.6893870360245429,
 'kappa': 0.3770724079241795,
 'matthews_cc': 0.37744214484785926,
 'confusion_matrix': [[[455, 196], [170, 361]]],
 'num_compounds': 1182}

## Neural Network with Fingerprint Split

In [34]:
response_col = "class"
compound_id = "compound_id"
smiles_col = "rdkit_smiles"
split_uuid = "64ffab14-47a7-44e8-bfa1-8576f640e241" # fingerprint
# Set up file locations 
dataset_file = '/home/jupyter/hERG_Analysis/herg_chembl_curated.csv' 
odir = '/home/jupyter/hERG_Analysis/Models' 

params = {
        "verbose": "True",
        "datastore": "False",
        "save_results": "False",
        "prediction_type": "classification",
        "dataset_key": dataset_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        # using previously split data
        "previously_split": "True",
        "split_uuid" : split_uuid,
        "split_only": "False",
        # not needed becuase already using splits established previously
        # "split_strategy": "train_valid_test",
        "splitter": "fingerprint",
        # "split_valid_frac": "0.10",
        # "split_test_frac": "0.10",
        "featurizer": "ecfp",
        "model_type": "NN",
        "verbose": "True",
        "transformers": "True",
        'max_epochs': '70',
        "rerun": "False",
        "result_dir": odir,
    #    "rf_estimators": 64,
     #   "rf_max_depth": 24,
      #  "rf_max_features": 16,
       # "model_choice_score_type": "recall"
        # "ampl_version": " "
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

2024-04-19 12:31:19,351 Previous dataset split restored


['/home/jupyter/hERG_Analysis/Models/herg_chembl_curated/NN_ecfp_fingerprint_classification/c763a0d5-6615-4365-98d1-959e0593c97b/model/checkpoint1.pt', '/home/jupyter/hERG_Analysis/Models/herg_chembl_curated/NN_ecfp_fingerprint_classification/c763a0d5-6615-4365-98d1-959e0593c97b/model/checkpoint2.pt', '/home/jupyter/hERG_Analysis/Models/herg_chembl_curated/NN_ecfp_fingerprint_classification/c763a0d5-6615-4365-98d1-959e0593c97b/model/checkpoint3.pt', '/home/jupyter/hERG_Analysis/Models/herg_chembl_curated/NN_ecfp_fingerprint_classification/c763a0d5-6615-4365-98d1-959e0593c97b/model/checkpoint4.pt']
/home/jupyter/hERG_Analysis/Models/herg_chembl_curated/NN_ecfp_fingerprint_classification/c763a0d5-6615-4365-98d1-959e0593c97b/model/checkpoint1.pt


In [35]:
pred_data_valid = pl.model_wrapper.get_perf_data(subset="valid", epoch_label="best")
pred_results_valid = pred_data_valid.get_prediction_results()
pred_results_valid

{'roc_auc_score': 0.695210694816994,
 'prc_auc_score': 0.7040109225382963,
 'cross_entropy': 0.6810761178413036,
 'model_choice_score': 0.695210694816994,
 'precision': 0.6350067842605156,
 'recall_score': 0.7370078740157481,
 'npv': 0.6238738738738738,
 'accuracy_score': 0.6308213378492803,
 'bal_accuracy': 0.6221669406708776,
 'kappa': 0.24757504288791277,
 'matthews_cc': 0.25150211928785704,
 'confusion_matrix': [[[277, 269], [167, 468]]],
 'num_compounds': 1181}

In [36]:
pred_data_test = pl.model_wrapper.get_perf_data(subset="test", epoch_label="best")
pred_results_test = pred_data_test.get_prediction_results()
pred_results_test

{'roc_auc_score': 0.5531028583943507,
 'prc_auc_score': 0.4590376185859878,
 'cross_entropy': 0.8434987584748118,
 'precision': 0.44212218649517687,
 'recall_score': 0.5705394190871369,
 'npv': 0.6310160427807486,
 'accuracy_score': 0.5316990701606086,
 'bal_accuracy': 0.5377661432097596,
 'kappa': 0.07225405705355425,
 'matthews_cc': 0.0743256192835665,
 'confusion_matrix': [[[354, 347], [207, 275]]],
 'num_compounds': 1183}

## NN with Random Split

In [37]:
response_col = "class"
compound_id = "compound_id"
smiles_col = "rdkit_smiles"
split_uuid = "ac01fe91-0793-4566-9340-2ff90e6a58c5" # random
# Set up file locations 
dataset_file = '/home/jupyter/hERG_Analysis/herg_chembl_curated.csv' 
odir = '/home/jupyter/hERG_Analysis/Models' 

params = {
        "verbose": "True",
        "datastore": "False",
        "save_results": "False",
        "prediction_type": "classification",
        "dataset_key": dataset_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        # using previously split data
        "previously_split": "True",
        "split_uuid" : split_uuid,
        "split_only": "False",
        # not needed becuase already using splits established previously
        # "split_strategy": "train_valid_test",
        "splitter": "random",
        # "split_valid_frac": "0.10",
        # "split_test_frac": "0.10",
        "featurizer": "ecfp",
        "model_type": "NN",
        "verbose": "True",
        "transformers": "True",
        'max_epochs': '70',
        "rerun": "False",
        "result_dir": "",
    #    "rf_estimators": 64,
     #   "rf_max_depth": 24,
      #  "rf_max_features": 16,
       # "model_choice_score_type": "recall"
        # "ampl_version": " "
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

2024-04-19 12:32:52,858 Previous dataset split restored


['herg_chembl_curated/NN_ecfp_random_classification/d2a56cc1-895d-4062-9f7b-8e7a9ff053f1/model/checkpoint1.pt', 'herg_chembl_curated/NN_ecfp_random_classification/d2a56cc1-895d-4062-9f7b-8e7a9ff053f1/model/checkpoint2.pt', 'herg_chembl_curated/NN_ecfp_random_classification/d2a56cc1-895d-4062-9f7b-8e7a9ff053f1/model/checkpoint3.pt', 'herg_chembl_curated/NN_ecfp_random_classification/d2a56cc1-895d-4062-9f7b-8e7a9ff053f1/model/checkpoint4.pt', 'herg_chembl_curated/NN_ecfp_random_classification/d2a56cc1-895d-4062-9f7b-8e7a9ff053f1/model/checkpoint5.pt']
herg_chembl_curated/NN_ecfp_random_classification/d2a56cc1-895d-4062-9f7b-8e7a9ff053f1/model/checkpoint1.pt


In [38]:
pred_data_valid = pl.model_wrapper.get_perf_data(subset="valid", epoch_label="best")
pred_results_valid = pred_data_valid.get_prediction_results()
pred_results_valid

{'roc_auc_score': 0.8344208832489636,
 'prc_auc_score': 0.8348802769749981,
 'cross_entropy': 1.2015749747192919,
 'model_choice_score': 0.8344208832489636,
 'precision': 0.7572156196943973,
 'recall_score': 0.7483221476510067,
 'npv': 0.7470489038785835,
 'accuracy_score': 0.7521150592216582,
 'bal_accuracy': 0.7521474219483703,
 'kappa': 0.5042443120492655,
 'matthews_cc': 0.5042796835069806,
 'confusion_matrix': [[[443, 143], [150, 446]]],
 'num_compounds': 1182}

In [39]:
pred_data_test = pl.model_wrapper.get_perf_data(subset="test", epoch_label="best")
pred_results_test = pred_data_test.get_prediction_results()
pred_results_test

{'roc_auc_score': 0.8131598728453717,
 'prc_auc_score': 0.8095288401381207,
 'cross_entropy': 1.368360832898366,
 'precision': 0.7243816254416962,
 'recall_score': 0.7269503546099291,
 'npv': 0.75,
 'accuracy_score': 0.7377326565143824,
 'bal_accuracy': 0.7372615850719548,
 'kappa': 0.4744496652381198,
 'matthews_cc': 0.4744523925143748,
 'confusion_matrix': [[[462, 156], [154, 410]]],
 'num_compounds': 1182}

In [40]:
response_col = "class"
compound_id = "compound_id"
smiles_col = "rdkit_smiles"
split_uuid = "b66b295b-92ba-47c5-8fc5-5cdb53849fee" # random
dataset_file = '/home/jupyter/hERG/final_herg_chembl.csv' -- CHANGE THIS
odir = '/home/jupyter/hERG/Models' -- CHANGE THIS

params = {
        "verbose": "True",
        "datastore": "False",
        "save_results": "False",
        "prediction_type": "classification",
        "dataset_key": dataset_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        # using previously split data
        "previously_split": "True",
        "split_uuid" : split_uuid,
        "split_only": "False",
        # not needed becuase already using splits established previously
        # "split_strategy": "train_valid_test",
        "splitter": "random",
        # "split_valid_frac": "0.10",
        # "split_test_frac": "0.10",
        "featurizer": "ecfp",
        "model_type": "NN",
        "verbose": "True",
        "transformers": "True",
        'max_epochs': '70',
        "rerun": "False",
        "result_dir": odir,
    #    "rf_estimators": 64,
     #   "rf_max_depth": 24,
      #  "rf_max_features": 16,
       # "model_choice_score_type": "recall"
        # "ampl_version": " "
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

SyntaxError: invalid syntax (1522650669.py, line 5)

In [ ]:
!tar -tf *.tar.gz

In [ ]:
!tar xzf *.tar.gz -C /tmp

In [ ]:
!cat /tmp/model_metadata.json | head

In [ ]:
import joblib
# load the model from disk
loaded_model = joblib.load("/tmp/best_model/model.joblib")
loaded_model

In [ ]:
import json
import pandas as pd
data = json.load(open('/tmp/model_metadata.json'))
# pd.DataFrame(data["model_parameters"])
data['model_parameters']

In [ ]:
pred_data_valid = pl.model_wrapper.get_perf_data(subset="valid", epoch_label="best")
pred_results_valid = pred_data_valid.get_prediction_results()
pred_results_valid

In [ ]:
pred_data_test = pl.model_wrapper.get_perf_data(subset="test", epoch_label="best")
pred_results_test = pred_data_test.get_prediction_results()
pred_results_test

# XGBOOST

## XGBOOST with Scaffold Split

In [41]:
response_col = "class"
compound_id = "compound_id"
smiles_col = "rdkit_smiles"
split_uuid = "5f834022-46ac-4c7b-bca1-9573f6eb940f" # scaffold
# Set up file locations 
dataset_file = '/home/jupyter/hERG_Analysis/herg_chembl_curated.csv' 
odir = '/home/jupyter/hERG_Analysis/Models'

params = {
        "verbose": "True",
        "datastore": "False",
        "save_results": "False",
        "prediction_type": "classification",
        "dataset_key": dataset_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        # using previously split data
        "previously_split": "True",
        "split_uuid" : split_uuid,
        "split_only": "False",
        # not needed becuase already using splits established previously
        # "split_strategy": "train_valid_test",
        "splitter": "scaffold",
        # "split_valid_frac": "0.10",
        # "split_test_frac": "0.10",
        "featurizer": "ecfp",
        "model_type": "xgboost",
        "verbose": "True",
        "transformers": "True",
        'max_epochs': '70',
        "rerun": "False",
        "result_dir": odir,
    #    "rf_estimators": 64,
     #   "rf_max_depth": 24,
      #  "rf_max_features": 16,
       # "model_choice_score_type": "recall"
        # "ampl_version": " "
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

2024-04-19 12:37:11,137 Previous dataset split restored


In [42]:
pred_data_valid = pl.model_wrapper.get_perf_data(subset="valid", epoch_label="best")
pred_results_valid = pred_data_valid.get_prediction_results()
pred_results_valid

{'roc_auc_score': 0.7428823955176321,
 'prc_auc_score': 0.7074451263959952,
 'cross_entropy': 0.5930257916789979,
 'model_choice_score': 0.7428823955176321,
 'precision': 0.672514619883041,
 'recall_score': 0.6377079482439926,
 'npv': 0.7070254110612855,
 'accuracy_score': 0.6920473773265652,
 'bal_accuracy': 0.6878087323123239,
 'kappa': 0.3771400279691124,
 'matthews_cc': 0.3775736539363787,
 'confusion_matrix': [[[473, 168], [196, 345]]],
 'num_compounds': 1182}

In [43]:
pred_data_test = pl.model_wrapper.get_perf_data(subset="test", epoch_label="best")
pred_results_test = pred_data_test.get_prediction_results()
pred_results_test

{'roc_auc_score': 0.7525319586555234,
 'prc_auc_score': 0.7045571867652014,
 'cross_entropy': 0.5865911441214937,
 'precision': 0.6686746987951807,
 'recall_score': 0.6271186440677966,
 'npv': 0.7105263157894737,
 'accuracy_score': 0.6928934010152284,
 'bal_accuracy': 0.68683121143482,
 'kappa': 0.3758150252690565,
 'matthews_cc': 0.37642153214757473,
 'confusion_matrix': [[[486, 165], [198, 333]]],
 'num_compounds': 1182}

## XGBOOST with Fingerprint Split

In [44]:
response_col = "class"
compound_id = "compound_id"
smiles_col = "rdkit_smiles"
split_uuid = "64ffab14-47a7-44e8-bfa1-8576f640e241" # fingerprint
# Set up file locations 
dataset_file = '/home/jupyter/hERG_Analysis/herg_chembl_curated.csv' 
odir = '/home/jupyter/hERG_Analysis/Models' 

params = {
        "verbose": "True",
        "datastore": "False",
        "save_results": "False",
        "prediction_type": "classification",
        "dataset_key": dataset_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        # using previously split data
        "previously_split": "True",
        "split_uuid" : split_uuid,
        "split_only": "False",
        # not needed becuase already using splits established previously
        # "split_strategy": "train_valid_test",
        "splitter": "fingerprint",
        # "split_valid_frac": "0.10",
        # "split_test_frac": "0.10",
        "featurizer": "ecfp",
        "model_type": "xgboost",
        "verbose": "True",
        "transformers": "True",
        'max_epochs': '70',
        "rerun": "False",
        "result_dir": odir,
    #    "rf_estimators": 64,
     #   "rf_max_depth": 24,
      #  "rf_max_features": 16,
       # "model_choice_score_type": "recall"
        # "ampl_version": " "
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

2024-04-19 12:37:29,908 Previous dataset split restored


In [45]:
pred_data_valid = pl.model_wrapper.get_perf_data(subset="valid", epoch_label="best")
pred_results_valid = pred_data_valid.get_prediction_results()
pred_results_valid

{'roc_auc_score': 0.6853869227884976,
 'prc_auc_score': 0.6917148998899232,
 'cross_entropy': 0.6456904332565345,
 'model_choice_score': 0.6853869227884976,
 'precision': 0.6650246305418719,
 'recall_score': 0.6377952755905512,
 'npv': 0.5979020979020979,
 'accuracy_score': 0.632514817950889,
 'bal_accuracy': 0.6320844509820888,
 'kappa': 0.2632902804807584,
 'matthews_cc': 0.26354708336476995,
 'confusion_matrix': [[[342, 204], [230, 405]]],
 'num_compounds': 1181}

In [46]:
pred_data_test = pl.model_wrapper.get_perf_data(subset="test", epoch_label="best")
pred_results_test = pred_data_test.get_prediction_results()
pred_results_test

{'roc_auc_score': 0.6135692342297016,
 'prc_auc_score': 0.4887840333977358,
 'cross_entropy': 0.6836502486273109,
 'precision': 0.48884381338742394,
 'recall_score': 0.5,
 'npv': 0.6507246376811594,
 'accuracy_score': 0.5832628909551987,
 'bal_accuracy': 0.5702567760342367,
 'kappa': 0.14001442109904105,
 'matthews_cc': 0.1400402042855608,
 'confusion_matrix': [[[449, 252], [241, 241]]],
 'num_compounds': 1183}

## XGBOOST with Random Split

In [3]:
response_col = "class"
compound_id = "compound_id"
smiles_col = "rdkit_smiles"
split_uuid = "ac01fe91-0793-4566-9340-2ff90e6a58c5" # random
# Set up file locations 
dataset_file = '/home/jupyter/hERG_Analysis/herg_chembl_curated.csv' 
odir = '/home/jupyter/hERG_Analysis/Models' 

params = {
        "verbose": "True",
        "datastore": "False",
        "save_results": "False",
        "prediction_type": "classification",
        "dataset_key": dataset_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        # using previously split data
        "previously_split": "True",
        "split_uuid" : split_uuid,
        "split_only": "False",
        # not needed becuase already using splits established previously
        # "split_strategy": "train_valid_test",
        "splitter": "random",
        # "split_valid_frac": "0.10",
        # "split_test_frac": "0.10",
        "featurizer": "ecfp",
        "model_type": "xgboost",
        "verbose": "True",
        "transformers": "True",
        'max_epochs': '70',
        "rerun": "False",
        "result_dir": odir,
    #    "rf_estimators": 64,
     #   "rf_max_depth": 24,
      #  "rf_max_features": 16,
       # "model_choice_score_type": "recall"
        # "ampl_version": " "
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

2024-05-01 13:35:43,465 Previous dataset split restored


In [4]:
pred_data_valid = pl.model_wrapper.get_perf_data(subset="valid", epoch_label="best")
pred_results_valid = pred_data_valid.get_prediction_results()
pred_results_valid

{'roc_auc_score': 0.8393356162814669,
 'prc_auc_score': 0.8398840075899756,
 'cross_entropy': 0.5066378439919058,
 'model_choice_score': 0.8393356162814669,
 'precision': 0.7590987868284229,
 'recall_score': 0.7348993288590604,
 'npv': 0.7388429752066116,
 'accuracy_score': 0.748730964467005,
 'bal_accuracy': 0.7488489818356736,
 'kappa': 0.49756262326428147,
 'matthews_cc': 0.49781984792870515,
 'confusion_matrix': [[[447, 139], [158, 438]]],
 'num_compounds': 1182}

In [5]:
pred_data_test = pl.model_wrapper.get_perf_data(subset="test", epoch_label="best")
pred_results_test = pred_data_test.get_prediction_results()
pred_results_test

{'roc_auc_score': 0.8143763340907525,
 'prc_auc_score': 0.8066943583053293,
 'cross_entropy': 0.5295807253638842,
 'precision': 0.7127659574468085,
 'recall_score': 0.7127659574468085,
 'npv': 0.7378640776699029,
 'accuracy_score': 0.7258883248730964,
 'bal_accuracy': 0.7253150175583557,
 'kappa': 0.4506300351167114,
 'matthews_cc': 0.4506300351167114,
 'confusion_matrix': [[[456, 162], [162, 402]]],
 'num_compounds': 1182}

# Performance of the Models
We evaluate model performance by measuring how a accurate models are on validation and test sets. The validation set is used while optimizing the model and choosing the best parameter settings. Then the performance on the test set is the final judge of model performance.

In [50]:
# Model Performance
# Before hyperparameterization optimization
from atomsci.ddm.pipeline import compare_models as cm
basic_pred_df = cm.get_filesystem_perf_results(odir, pred_type='classification')

2024-04-19 12:38:24,654 Warning: column names have been changed to align with get_multitask_perf_from_tracker(): featurizer is now features and <subset>_<metric> has been changed to best_<subset>_<metric>.


Found data for 9 models under /home/jupyter/hERG_Analysis/Models


The pred_df dataframe has details about the model_uuid, model_path, ampl_version, model_type, features, splitter and the results for popular metrics that help evaluate the performance. Let us view the contents of the pred_df dataframe.

In [51]:
# save pred_df
import os
basic_pred_df.to_csv(os.path.join(odir, 'basic_pred_df.csv'))

In [52]:
# View the pred_df dataframe
basic_pred_df

,model_uuid,model_path,ampl_version,model_type,dataset_key,features,splitter,split_strategy,model_score_type,feature_transform_type,...,rf_max_depth,xgb_gamma,xgb_learning_rate,xgb_max_depth,xgb_colsample_bytree,xgb_subsample,xgb_n_estimators,xgb_min_child_weight,model_parameters_dict,feat_parameters_dict
3,e79bddbf-6ca1-4756-b81e-e9135481fa9c,/home/jupyter/hERG_Analysis/Models/herg_chembl...,1.6.0,RF,/home/jupyter/hERG_Analysis/herg_chembl_curate...,ecfp,random,train_valid_test,roc_auc,normalization,...,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""rf_estimators"": 64, ""rf_max_depth"": 24, ""rf_...","{""ecfp_radius"": 2, ""ecfp_size"": 1024}"
2,ba8a6514-e5cc-4b5a-b271-658d589050c1,/home/jupyter/hERG_Analysis/Models/herg_chembl...,1.6.0,xgboost,/home/jupyter/hERG_Analysis/herg_chembl_curate...,ecfp,random,train_valid_test,roc_auc,normalization,...,NaN,0.0,0.1,6.0,1.0,1.0,100.0,1.0,"{""xgb_colsample_bytree"": 1.0, ""xgb_gamma"": 0.0...","{""ecfp_radius"": 2, ""ecfp_size"": 1024}"
6,08f8417a-642c-4ee1-8e57-72ad410518cc,/home/jupyter/hERG_Analysis/Models/herg_chembl...,1.6.0,RF,/home/jupyter/hERG_Analysis/herg_chembl_curate...,ecfp,scaffold,train_valid_test,roc_auc,normalization,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""rf_estimators"": 500, ""rf_max_depth"": null, ""...","{""ecfp_radius"": 2, ""ecfp_size"": 1024}"
5,804a2c09-754c-4dc4-986f-273eea98a9be,/home/jupyter/hERG_Analysis/Models/herg_chembl...,1.6.0,RF,/home/jupyter/hERG_Analysis/herg_chembl_curate...,ecfp,scaffold,train_valid_test,roc_auc,normalization,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""rf_estimators"": 500, ""rf_max_depth"": null, ""...","{""ecfp_radius"": 2, ""ecfp_size"": 1024}"
1,5de4bfcc-33c9-490b-b02e-9745fe59d0d4,/home/jupyter/hERG_Analysis/Models/herg_chembl...,1.6.0,NN,/home/jupyter/hERG_Analysis/herg_chembl_curate...,ecfp,scaffold,train_valid_test,roc_auc,normalization,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""best_epoch"": 67, ""dropouts"": [0.4, 0.4], ""la...","{""ecfp_radius"": 2, ""ecfp_size"": 1024}"
4,87273377-c53f-4210-a1a0-5ae7dda4fbaf,/home/jupyter/hERG_Analysis/Models/herg_chembl...,1.6.0,RF,/home/jupyter/hERG_Analysis/herg_chembl_curate...,ecfp,fingerprint,train_valid_test,roc_auc,normalization,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""rf_estimators"": 500, ""rf_max_depth"": null, ""...","{""ecfp_radius"": 2, ""ecfp_size"": 1024}"
7,d45bae4e-5d99-4080-8bca-a3fb35a51d3f,/home/jupyter/hERG_Analysis/Models/herg_chembl...,1.6.0,xgboost,/home/jupyter/hERG_Analysis/herg_chembl_curate...,ecfp,scaffold,train_valid_test,roc_auc,normalization,...,NaN,0.0,0.1,6.0,1.0,1.0,100.0,1.0,"{""xgb_colsample_bytree"": 1.0, ""xgb_gamma"": 0.0...","{""ecfp_radius"": 2, ""ecfp_size"": 1024}"
0,c763a0d5-6615-4365-98d1-959e0593c97b,/home/jupyter/hERG_Analysis/Models/herg_chembl...,1.6.0,NN,/home/jupyter/hERG_Analysis/herg_chembl_curate...,ecfp,fingerprint,train_valid_test,roc_auc,normalization,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""best_epoch"": 9, ""dropouts"": [0.4, 0.4], ""lay...","{""ecfp_radius"": 2, ""ecfp_size"": 1024}"
8,4bbdcd2c-7f75-47d7-a670-cb4e14ede471,/home/jupyter/hERG_Analysis/Models/herg_chembl...,1.6.0,xgboost,/home/jupyter/hERG_Analysis/herg_chembl_curate...,ecfp,fingerprint,train_valid_test,roc_auc,normalization,...,NaN,0.0,0.1,6.0,1.0,1.0,100.0,1.0,"{""xgb_colsample_bytree"": 1.0, ""xgb_gamma"": 0.0...","{""ecfp_radius"": 2, ""ecfp_size"": 1024}"


In [55]:
# get columns of the file
basic_pred_df.columns

Index(['model_uuid', 'model_path', 'ampl_version', 'model_type', 'dataset_key',
       'features', 'splitter', 'split_strategy', 'model_score_type',
       'feature_transform_type', 'model_choice_score',
       'best_train_roc_auc_score', 'best_train_prc_auc_score',
       'best_train_precision', 'best_train_recall_score',
       'best_train_num_compounds', 'best_train_accuracy_score',
       'best_train_bal_accuracy', 'best_train_npv', 'best_train_matthews_cc',
       'best_train_kappa', 'best_train_cross_entropy',
       'best_train_confusion_matrix', 'best_valid_roc_auc_score',
       'best_valid_prc_auc_score', 'best_valid_precision',
       'best_valid_recall_score', 'best_valid_num_compounds',
       'best_valid_accuracy_score', 'best_valid_bal_accuracy',
       'best_valid_npv', 'best_valid_matthews_cc', 'best_valid_kappa',
       'best_valid_cross_entropy', 'best_valid_confusion_matrix',
       'best_test_roc_auc_score', 'best_test_prc_auc_score',
       'best_test_precision', 

# Top Performing Model
To pick the top performing model, we sort the best_valid_auc_score column in descending order and pick the one that is maximum.

In [59]:
# Top performing model
top_basic_model=basic_pred_df.sort_values(by="best_valid_roc_auc_score", ascending=False).iloc[0]
top_basic_model

model_uuid                            e79bddbf-6ca1-4756-b81e-e9135481fa9c
model_path               /home/jupyter/hERG_Analysis/Models/herg_chembl...
ampl_version                                                         1.6.0
model_type                                                              RF
dataset_key              /home/jupyter/hERG_Analysis/herg_chembl_curate...
                                               ...                        
xgb_subsample                                                          NaN
xgb_n_estimators                                                       NaN
xgb_min_child_weight                                                   NaN
model_parameters_dict    {"rf_estimators": 64, "rf_max_depth": 24, "rf_...
feat_parameters_dict                 {"ecfp_radius": 2, "ecfp_size": 1024}
Name: 3, Length: 64, dtype: object

# Model Tarball
The model_path or the location of the tarball where the top performing model is saved is in top_model.model_path.

In [60]:
# Top performing model path
top_basic_model.model_path

'/home/jupyter/hERG_Analysis/Models/herg_chembl_curated_model_e79bddbf-6ca1-4756-b81e-e9135481fa9c.tar.gz'